In [197]:
import re
from bs4 import BeautifulSoup
import json 

In [198]:
def get_indexes(t,token):
    '''
    returns all the indexes of a term in a text
    '''
    indexes = []
    old = 0
    while(t.find(token) != -1) :
        index = old + t.find(token)
        indexes.append(index)
        old = t.find(token)+len(token)
        t = t[t.find(token)+len(token):len(t)]
    return indexes



In [215]:
def write_json(data,file):
    with open(file.replace('txt','json'), 'w') as outfile:
        for d in data:
            json.dump(d, outfile)

In [216]:
def parse_file(texts,file):
    '''
    returns the data paresed
    '''

    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;-_.]')
    BAD_SYMBOLS_RE = re.compile('[\t\n]')
    data = []
    for text in texts:
        idx = text.split('\t')[0]
        text = text.replace(text.split('\t')[0], ' ')
        text = BAD_SYMBOLS_RE.sub(' ',text)

        text = text.replace("<category=", "<category type =")
        soup = BeautifulSoup(text, 'lxml')

        cat_done = []
        labels = []
        for cat in soup.find_all('category'):
            if cat.text not in cat_done :
                indexes = get_indexes(soup.get_text()," "+cat.text+" ")
                if soup.get_text().split()[0] == cat.text :
                    labels.append([0,len(cat.text),cat.attrs['type']])
                if soup.get_text().split()[-1] == cat.text :
                    labels.append([len(text)-len(cat.text),len(text),cat.attrs['type']])
                for i in indexes :
                    labels.append([i+1,i+1+len(cat.text),cat.attrs['type']])
                cat_done.append(cat.text)

        data.append({'id':idx,'text' : soup.get_text() ,'labels' : labels})
    return data

In [217]:
import os 
files = ['datasets/'+f for f in os.listdir('datasets/') if 'txt' in f]
for file in files :
    
    f = open(file)
    texts = f.readlines()
    data = parse_file(texts,file)
    write_json(data,file)
    

## Exploratory study 

In [218]:
def explore(texts,file):
    types = []
    names = []
    for text in texts:
        text = text.replace("<category=", "<category type =")
        soup = BeautifulSoup(text, 'lxml')
        for cat in soup.find_all('category'):
            names.append(cat.text)
            types.append(cat.attrs['type'])
    return (types,names)

In [237]:
import pandas as pd 
import os 
files = ['datasets/'+f for f in os.listdir('datasets/') if 'txt' in f]
for file in files :
    
    f = open(file)
    texts = f.readlines()
    types,names = explore(texts,file)
    df = pd.DataFrame({'types':types,'names':names})
    print(file)
    print('types___________')
    print("Types number : ",len(df.types.value_counts()))
    print(df.types.value_counts())
    print('names____________')
    print("Names number : ",len(df.names.value_counts()))

    print(df.names.value_counts())
    
    

datasets/NCBI_corpus_training.txt
types___________
Types number :  4
SpecificDisease     2959
Modifier            1292
DiseaseClass         781
CompositeMention     116
Name: types, dtype: int64
names____________
Names number :  1710
DM                                              120
DMD                                             109
APC                                              91
ALD                                              86
PWS                                              75
                                               ... 
hepatic encephalopathy                            1
angiokeratoma corporis diffusum                   1
familial neurohypophyseal diabetes insipidus      1
Type I human complement C2 deficiency             1
cancer-causing                                    1
Name: names, Length: 1710, dtype: int64
datasets/NCBI_corpus_development.txt
types___________
Types number :  4
SpecificDisease     409
Modifier            218
DiseaseClass        127
CompositeM